In [5]:
from os import read
import torch
from lietorch import SO3, SE3, LieGroupParameter

import argparse
import numpy as np
import time
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import chamfer3D.dist_chamfer_3D
# import pytorch3d
from pdb import post_mortem, set_trace as bp

import itertools
from torch.autograd import Variable
import random
import torch.nn.functional as FF
from sklearn.cluster import kmeans_plusplus
import open3d as o3d
import os

In [6]:
def read_obj(obj_path,for_open_mesh=False):
    with open(obj_path) as file:
        flag = 0
        points = []
        normals = []
        faces = []
        while 1:
            line = file.readline()
            if not line:
                break
            strs = line.split(" ")
            if strs[0] =='o' and flag==0:
                flag = 1
                continue
            elif strs[0]=='o' and flag==1:
                break
            if strs[0] =='v':
                points.append((float(strs[1]),float(strs[2]),float(strs[3])))
            
            if strs[0] == 'vn':
                normals.append((float(strs[1]),float(strs[2]),float(strs[3])))
            if strs[0] =='f':
                single_line_face = strs[1:]
                
#                 for sf in single_line_face:
#                     face_index = sf.split('/')
#                     faces.append((int(face_index[0]),int(face_index[1]),int(face_index[2])))
#                     break
                f_co = []
                for sf in single_line_face:
                    face_tmp = sf.split('/')[0]
                    f_co.append(face_tmp)
                if for_open_mesh == False:
                    if len(f_co)==3:
                        faces.append((int(f_co[0]),int(f_co[1]),int(f_co[2])))
                    elif len(f_co)==4:
                        faces.append((int(f_co[0]),int(f_co[1]),int(f_co[2])))
                        faces.append((int(f_co[0]),int(f_co[1]),int(f_co[3])))
                        faces.append((int(f_co[1]),int(f_co[2]),int(f_co[3])))
                        faces.append((int(f_co[0]),int(f_co[3]),int(f_co[2])))
                else:
                    faces.append([int(ver) for ver in f_co])
                
    points = np.array(points)

    normals = np.array(normals)
    faces = np.array(faces)
    return points,normals,faces



In [7]:
class LaplacianLoss(nn.Module):
    def __init__(self, vertex, faces, average=False):
        super(LaplacianLoss, self).__init__()
        self.nv = vertex.shape[0]
        self.nf = faces.shape[0]
        self.average = average
        laplacian = np.zeros([self.nv, self.nv]).astype(np.float32)

        laplacian[faces[:, 0], faces[:, 1]] = -1
        laplacian[faces[:, 1], faces[:, 0]] = -1
        laplacian[faces[:, 1], faces[:, 2]] = -1
        laplacian[faces[:, 2], faces[:, 1]] = -1
        laplacian[faces[:, 2], faces[:, 0]] = -1
        laplacian[faces[:, 0], faces[:, 2]] = -1

        r, c = np.diag_indices(laplacian.shape[0])
        laplacian[r, c] = -laplacian.sum(1)

        
        for i in range(self.nv):
            if laplacian[i, i]!=0: 
                laplacian[i, :] /= laplacian[i, i]

        self.register_buffer('laplacian', torch.from_numpy(laplacian))

    def forward(self, x):
        batch_size = x.shape[0]
        x = torch.matmul(self.laplacian.cuda(), x)
        dims = tuple(range(x.ndimension())[1:])
        x = x.pow(2).sum(dims)
        if self.average:
            return x.sum() / batch_size
        else:
            return x

In [4]:
if __name__ == '__main__':
    # parser = argparse.ArgumentParser()
    # parser.add_argument('--old', default='frame_000001.obj')
    # parser.add_argument('--target', default='frame_000002.obj')
    # args = parser.parse_args()

    # output_path = args.problem.replace('.g2o', '_rotavg.g2o')
    # input_pose_graph = read_g2o(args.problem)

    seed = 2000
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    points_info,normals_info,face_info = read_obj('frame_000001.obj')
    old_ones = np.ones((points_info.shape[0],1))
    
    
    points_info = np.concatenate((points_info,old_ones),axis=1)
    all_info_list = []
    for i in range(20, 30):
        path = "frame_{:06d}.obj".format(i+1)

        tmp_target = read_obj(path)[0]
        target_ones = np.ones((tmp_target.shape[0],1))
        tmp_target_new = np.concatenate((tmp_target,target_ones),axis=1)
        all_info_list.append(tmp_target_new)

        
    la_loss = LaplacianLoss(torch.tensor(points_info),torch.tensor(face_info-1),average=True)
    all_info = np.stack(all_info_list)
    old_info = points_info
    target_info = all_info
    # W,R= train(old_info=points_info,target_info=all_info)

    N = points_info.shape[0]
    F = target_info.shape[0]
    B = 40  ### 这个肯定可以通过读pkl得到

    center, indexes = kmeans_plusplus(old_info[:, :-1], n_clusters=B, random_state=0)
    old_mesh = torch.tensor(old_info).float().cuda().detach()
    # bp()
    # ones = torch.ones(old_mesh.shape[0], 1).cuda()
    # x = torch.cat([old_mesh, ones], dim=1)

    center_t = torch.tensor(center).repeat(F, 1, 1).cuda()
    target_mesh = torch.tensor(target_info, requires_grad=False).float().cuda().detach()

    # bp()
    # W = torch.randint(10,(N,B))
    # W = torch.randn((N,B)).cuda()
    # W = np.random.randint(10,size=(N,B))
    # bp()
    # W = SO3(torch.from_numpy(W).float().cuda())
    # W = torch.zeros((N, B), dtype=torch.float, requires_grad=True, device="cuda")
    W = torch.randn((N, B), requires_grad=True, device="cuda")
    W2 = torch.randn((N, B), requires_grad=True, device="cuda")
    # W = torch.tensor(W0.float(), requires_grad=True, device="cuda")
    # W = W * 0
    # bp()
    # W = LieGroupParameter(W)
    # bp()
    # R = torch.randint(10,(B,3,3))

    # bp()pcd.points = Vector3dVector(np_points)
    # random quaternion
   # bounding_box_max = old_mesh.max()
    # bounding_box_min = old_mesh.min()
    p1 = torch.randn((F, B, 7), requires_grad=True, device="cuda")
    # with torch.no_grad():
    #     p1[:, :, 4:] = center_t
    # W = W.cuda()
    # p1 = p1.cuda()
    # p1 = p1 / p1.norm(dim=-1, keepdim=True)
    # create SO3 object from quaternion (differentiable w.r.t q)
    
    # 4x4 transformation matrix (differentiable w.r.t R)
    

    # W = torch.tensor(np.load('weight.npy')).float().cuda()
    optimizer = optim.Adam([W,  p1],lr=1e-2)

    chamLoss = chamfer3D.dist_chamfer_3D.chamfer_3DDist()
    for i in range(10000):
        # if i == 2000:
        #     optimizer = optim.Adam([W, p1],lr=1e-3)
        optimizer.zero_grad()
        loss = 0
        loss2 = 0
        f_idxs = random.sample(range(F), F)
        
        for f in f_idxs:
            T = torch.randn(B, 4, 4)
            W1 = FF.softmax(W * 10)
            W1 = (W1 / (W1.sum(1, keepdim=True).detach()))
            # W3 = FF.softmax(W2)
            # W3 = (W3 / (W3.sum(1, keepdim=True).detach()))
            # dist= W3.unsqueeze(2) * old_mesh.unsqueeze(1).repeat(1, 45, 1)
            # dist = dist.sum(0)
            
            for b in range(B):
                # R = SO3.InitFromVec(p1[f, b])
                # bp()
                # bp()
                # with torch.no_grad():
                # temp = torch.cat([p1[f, b], dist[f][:-1]])
                R = SE3.InitFromVec(p1[f, b])
                # bp()
                T[b] = R.matrix()
            T = T.cuda()     
            # bp()      
            # bp() 
            # W1 = W
            # W1 = (W - W.min()) / (W.max() - W.min())
            
            # bp()
            # p1[:, :, 4:] = 
            # bp()
            # ones = torch.ones(old_mesh.shape[0], 1).cuda()
            # x = torch.cat([old_mesh, ones], dim=1)
            x = old_mesh
        
        
            bx = (T@x.T).permute(2, 0, 1)
            wbx = W1.unsqueeze(2) * bx
            # bp()
            wbx =  wbx.permute((1,0,2))
            # print("Wbx", wbx.shape)
            # wbx = wbx.sum(0, keepdim=True)[:, :, :-1]
            wbx = wbx.sum(0, keepdim=True)
            # loss_init = chamLoss(target_mesh[f][None],wbx)
            # loss += loss_init[0].sum()

#             bp()
            loss += torch.sum((target_mesh[f]-wbx)**2)
            # loss2 += la_loss(W1) * 100
            # loss += torch.sum((target_mesh[f]-wbx)**2) +la_loss(W.detach().cpu())
#             loss.backward()
            # print("loss",loss)
            # bp()
        # bp()
        (loss + loss2).backward()
        # print("this is loss for epoch {} : {} ".format(i,loss.detach().cpu().numpy()))
        print("this is loss for epoch {} : {}, {} ".format(i,loss / F, loss2/F))
        # print("p1: ", p1)
        optimizer.step()


    new_R = SE3.InitFromVec(p1)
    R = new_R.matrix()

    # W1 = (W - W.min()) / (W.max() - W.min())
    # W1 = (W1 / (W1.sum(1, keepdim=True).detach()))
    # bp()
    # return R.detach(), W1.detach()

/home/ivenwu/miniconda3/envs/lietorch/lib/python3.7/site-packages/ipykernel_launcher.py:94: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


this is loss for epoch 0 : 10621.0302734375, 0.0 
this is loss for epoch 1 : 10201.5625, 0.0 
this is loss for epoch 2 : 9797.103515625, 0.0 
this is loss for epoch 3 : 9407.7861328125, 0.0 
this is loss for epoch 4 : 9033.6826171875, 0.0 
this is loss for epoch 5 : 8674.6005859375, 0.0 
this is loss for epoch 6 : 8330.240234375, 0.0 
this is loss for epoch 7 : 8000.20654296875, 0.0 
this is loss for epoch 8 : 7684.0009765625, 0.0 
this is loss for epoch 9 : 7381.02294921875, 0.0 
this is loss for epoch 10 : 7090.6494140625, 0.0 
this is loss for epoch 11 : 6812.24169921875, 0.0 
this is loss for epoch 12 : 6545.09912109375, 0.0 
this is loss for epoch 13 : 6288.43115234375, 0.0 
this is loss for epoch 14 : 6041.42041015625, 0.0 
this is loss for epoch 15 : 5803.32373046875, 0.0 
this is loss for epoch 16 : 5573.54541015625, 0.0 
this is loss for epoch 17 : 5351.662109375, 0.0 
this is loss for epoch 18 : 5137.42724609375, 0.0 
this is loss for epoch 19 : 4930.75048828125, 0.0 
this is

this is loss for epoch 158 : 36.410213470458984, 0.0 
this is loss for epoch 159 : 35.95897674560547, 0.0 
this is loss for epoch 160 : 35.51527786254883, 0.0 
this is loss for epoch 161 : 35.07224655151367, 0.0 
this is loss for epoch 162 : 34.636226654052734, 0.0 
this is loss for epoch 163 : 34.21841049194336, 0.0 
this is loss for epoch 164 : 33.81858825683594, 0.0 
this is loss for epoch 165 : 33.42978286743164, 0.0 
this is loss for epoch 166 : 33.052490234375, 0.0 
this is loss for epoch 167 : 32.68193435668945, 0.0 
this is loss for epoch 168 : 32.31393051147461, 0.0 
this is loss for epoch 169 : 31.951309204101562, 0.0 
this is loss for epoch 170 : 31.60064697265625, 0.0 
this is loss for epoch 171 : 31.267438888549805, 0.0 
this is loss for epoch 172 : 30.961530685424805, 0.0 
this is loss for epoch 173 : 30.686193466186523, 0.0 
this is loss for epoch 174 : 30.429296493530273, 0.0 
this is loss for epoch 175 : 30.174530029296875, 0.0 
this is loss for epoch 176 : 29.91326904

this is loss for epoch 311 : 14.613351821899414, 0.0 
this is loss for epoch 312 : 14.560078620910645, 0.0 
this is loss for epoch 313 : 14.506464004516602, 0.0 
this is loss for epoch 314 : 14.45256519317627, 0.0 
this is loss for epoch 315 : 14.398126602172852, 0.0 
this is loss for epoch 316 : 14.343317031860352, 0.0 
this is loss for epoch 317 : 14.288042068481445, 0.0 
this is loss for epoch 318 : 14.23188304901123, 0.0 
this is loss for epoch 319 : 14.175396919250488, 0.0 
this is loss for epoch 320 : 14.11927318572998, 0.0 
this is loss for epoch 321 : 14.063422203063965, 0.0 
this is loss for epoch 322 : 14.008699417114258, 0.0 
this is loss for epoch 323 : 13.954690933227539, 0.0 
this is loss for epoch 324 : 13.902055740356445, 0.0 
this is loss for epoch 325 : 13.853045463562012, 0.0 
this is loss for epoch 326 : 13.801360130310059, 0.0 
this is loss for epoch 327 : 13.747629165649414, 0.0 
this is loss for epoch 328 : 13.69444751739502, 0.0 
this is loss for epoch 329 : 13.

this is loss for epoch 464 : 9.098176956176758, 0.0 
this is loss for epoch 465 : 9.078062057495117, 0.0 
this is loss for epoch 466 : 9.057596206665039, 0.0 
this is loss for epoch 467 : 9.037481307983398, 0.0 
this is loss for epoch 468 : 9.017661094665527, 0.0 
this is loss for epoch 469 : 8.998431205749512, 0.0 
this is loss for epoch 470 : 8.979658126831055, 0.0 
this is loss for epoch 471 : 8.961170196533203, 0.0 
this is loss for epoch 472 : 8.942598342895508, 0.0 
this is loss for epoch 473 : 8.92423152923584, 0.0 
this is loss for epoch 474 : 8.906206130981445, 0.0 
this is loss for epoch 475 : 8.88769245147705, 0.0 
this is loss for epoch 476 : 8.869561195373535, 0.0 
this is loss for epoch 477 : 8.85083293914795, 0.0 
this is loss for epoch 478 : 8.831855773925781, 0.0 
this is loss for epoch 479 : 8.812976837158203, 0.0 
this is loss for epoch 480 : 8.79391098022461, 0.0 
this is loss for epoch 481 : 8.775126457214355, 0.0 
this is loss for epoch 482 : 8.756684303283691, 0.

...

this is loss for epoch 5117 : 0.19196335971355438, 0.0 
this is loss for epoch 5118 : 0.19299927353858948, 0.0 
this is loss for epoch 5119 : 0.19446128606796265, 0.0 
this is loss for epoch 5120 : 0.19505201280117035, 0.0 
this is loss for epoch 5121 : 0.19430720806121826, 0.0 
this is loss for epoch 5122 : 0.19274453818798065, 0.0 
this is loss for epoch 5123 : 0.19130365550518036, 0.0 
this is loss for epoch 5124 : 0.19061143696308136, 0.0 
this is loss for epoch 5125 : 0.190757617354393, 0.0 
this is loss for epoch 5126 : 0.19137214124202728, 0.0 
this is loss for epoch 5127 : 0.19190572202205658, 0.0 
this is loss for epoch 5128 : 0.19193752110004425, 0.0 
this is loss for epoch 5129 : 0.19136443734169006, 0.0 
this is loss for epoch 5130 : 0.19035834074020386, 0.0 
this is loss for epoch 5131 : 0.18966440856456757, 0.0 
this is loss for epoch 5132 : 0.18965663015842438, 0.0 
this is loss for epoch 5133 : 0.18985429406166077, 0.0 
this is loss for epoch 5134 : 0.18990498781204224,

this is loss for epoch 5265 : 0.1775362491607666, 0.0 
this is loss for epoch 5266 : 0.17724238336086273, 0.0 
this is loss for epoch 5267 : 0.17704857885837555, 0.0 
this is loss for epoch 5268 : 0.17695216834545135, 0.0 
this is loss for epoch 5269 : 0.17693106830120087, 0.0 
this is loss for epoch 5270 : 0.17694735527038574, 0.0 
this is loss for epoch 5271 : 0.17696422338485718, 0.0 
this is loss for epoch 5272 : 0.1769586056470871, 0.0 
this is loss for epoch 5273 : 0.1769271194934845, 0.0 
this is loss for epoch 5274 : 0.17687687277793884, 0.0 
this is loss for epoch 5275 : 0.17681443691253662, 0.0 
this is loss for epoch 5276 : 0.17673994600772858, 0.0 
this is loss for epoch 5277 : 0.1766493022441864, 0.0 
this is loss for epoch 5278 : 0.17654214799404144, 0.0 
this is loss for epoch 5279 : 0.17642365396022797, 0.0 
this is loss for epoch 5280 : 0.1763039082288742, 0.0 
this is loss for epoch 5281 : 0.17619089782238007, 0.0 
this is loss for epoch 5282 : 0.1760888546705246, 0.0

this is loss for epoch 5413 : 0.179814875125885, 0.0 
this is loss for epoch 5414 : 0.1746210902929306, 0.0 
this is loss for epoch 5415 : 0.1689480096101761, 0.0 
this is loss for epoch 5416 : 0.16649405658245087, 0.0 
this is loss for epoch 5417 : 0.16778935492038727, 0.0 
this is loss for epoch 5418 : 0.1705242395401001, 0.0 
this is loss for epoch 5419 : 0.1719031184911728, 0.0 
this is loss for epoch 5420 : 0.17070718109607697, 0.0 
this is loss for epoch 5421 : 0.16806276142597198, 0.0 
this is loss for epoch 5422 : 0.16599102318286896, 0.0 
this is loss for epoch 5423 : 0.16568289697170258, 0.0 
this is loss for epoch 5424 : 0.166696235537529, 0.0 
this is loss for epoch 5425 : 0.16764666140079498, 0.0 
this is loss for epoch 5426 : 0.1675303876399994, 0.0 
this is loss for epoch 5427 : 0.16642698645591736, 0.0 
this is loss for epoch 5428 : 0.16518737375736237, 0.0 
this is loss for epoch 5429 : 0.16458198428153992, 0.0 
this is loss for epoch 5430 : 0.16471199691295624, 0.0 
t

this is loss for epoch 5561 : 0.15936200320720673, 0.0 
this is loss for epoch 5562 : 0.15642622113227844, 0.0 
this is loss for epoch 5563 : 0.15505102276802063, 0.0 
this is loss for epoch 5564 : 0.15517637133598328, 0.0 
this is loss for epoch 5565 : 0.15629112720489502, 0.0 
this is loss for epoch 5566 : 0.1576739400625229, 0.0 
this is loss for epoch 5567 : 0.158675417304039, 0.0 
this is loss for epoch 5568 : 0.15884384512901306, 0.0 
this is loss for epoch 5569 : 0.15816648304462433, 0.0 
this is loss for epoch 5570 : 0.1569076031446457, 0.0 
this is loss for epoch 5571 : 0.15554200112819672, 0.0 
this is loss for epoch 5572 : 0.15445521473884583, 0.0 
this is loss for epoch 5573 : 0.15385417640209198, 0.0 
this is loss for epoch 5574 : 0.15373791754245758, 0.0 
this is loss for epoch 5575 : 0.15396051108837128, 0.0 
this is loss for epoch 5576 : 0.15431438386440277, 0.0 
this is loss for epoch 5577 : 0.15459579229354858, 0.0 
this is loss for epoch 5578 : 0.15467020869255066, 0

KeyboardInterrupt: 

In [ ]:
if __name__ == '__main__':
    seed = 2000
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    points_info,normals_info,face_info = read_obj('frame_000001.obj')
    old_ones = np.ones((points_info.shape[0],1))

    points_info = np.concatenate((points_info,old_ones),axis=1)

    old_info = points_info
    
    all_info_list = []
    for i in range(80,90):
        path = "frame_0000{}.obj".format(i+1)

        tmp_target = read_obj(path)[0]
        target_ones = np.ones((tmp_target.shape[0],1))
        tmp_target_new = np.concatenate((tmp_target,target_ones),axis=1)
        all_info_list.append(tmp_target_new)

    all_info = np.stack(all_info_list)
    old_info = points_info
    target_info = all_info
    # W,R= train(old_info=points_info,target_info=all_info)
#     bp()
    N = points_info.shape[0]
    F = target_info.shape[0]
    B = 40  ### 这个肯定可以通过读pkl得到

    center, indexes = kmeans_plusplus(old_info[:, :-1], n_clusters=B, random_state=0)
    old_mesh = torch.tensor(old_info).float().cuda().detach()
    # bp()
    # ones = torch.ones(old_mesh.shape[0], 1).cuda()
    # x = torch.cat([old_mesh, ones], dim=1)

    center_t = torch.tensor(center).repeat(F, 1, 1).cuda()
    target_mesh = torch.tensor(target_info, requires_grad=False).float().cuda().detach()

    
    
    p1 = torch.randn((F, B, 7), requires_grad=True, device="cuda")

#     W1 = FF.softmax(W * 10).clone().detach()
#     W1 = (W1 / (W1.sum(1, keepdim=True).detach()))

    optimizer = optim.Adam([p1],lr=1e-2)
    chamLoss = chamfer3D.dist_chamfer_3D.chamfer_3DDist()
#     W = torch.tensor(W)
    
#     W1 = np.load('weight.npy')
    W1 = torch.tensor(W1).cuda()
    la_loss = LaplacianLoss(torch.tensor(points_info),torch.tensor(face_info-1),average=True)
    
    for i in range(10000):
        if i == 400:
            optimizer = optim.Adam([p1],lr=1e-3)
        optimizer.zero_grad()
        loss = 0
        f_idxs = random.sample(range(F), F)
        
        for f in f_idxs:
            T = torch.randn(B, 4, 4)

            for b in range(B):
                # R = SO3.InitFromVec(p1[f, b])
                # bp()
                # bp()
                # with torch.no_grad():
                # temp = torch.cat([p1[f, b], dist[f][:-1]])
                R = SE3.InitFromVec(p1[f, b])
                # bp()
                T[b] = R.matrix()
            T = T.cuda()     
        
            x = old_mesh
        
        
            bx = (T@x.T).permute(2, 0, 1)
            wbx = W1.unsqueeze(2) * bx
            # bp()
            wbx =  wbx.permute((1,0,2))40
            # print("Wbx", wbx.shape)
            # wbx = wbx.sum(0, keepdim=True)[:, :, :-1]
            wbx = wbx.sum(0, keepdim=True)
            # loss_init = chamLoss(target_mesh[f][None],wbx)
            # loss += loss_init[0].sum()
#             bp()
            loss += torch.sum((target_mesh[f]-wbx)**2) 
#             print(loss)
        # bp()
        loss.backward()
        # print("this is loss for epoch {} : {} ".format(i,loss.detach().cpu().numpy()))
        print("this is loss for epoch {} : {} ".format(i,loss / F))
        # print("p1: ", p1)
        optimizer.step()


    new_R = SE3.InitFromVec(p1)
    R = new_R.matrix()

    # W1 = (W - W.min()) / (W.max() - W.min())
    # W1 = (W1 / (W1.sum(1, keepdim=True).detach()))
    # bp()
    # return R.detach(), W1.detach()

In [30]:
la_loss = LaplacianLoss(points_info,points_face-1,average=True)

In [31]:
la_loss(torch.tensor(wbx_result[-1][:, :-1]))

tensor(0.0052)

In [28]:
ccc = torch.tensor([[1,2,3],[4,5,6]])
ccc.size(0)

2

In [29]:
points_info.shape

(3199, 4)

In [38]:
points_info,points_normal,points_face = read_obj('frame_000001.obj')
# all_info_list = []

# for i in range(1, 9):
#     path = "frame_00000{}.obj".format(i+1)
#     all_info_list.append(read_obj(path))

In [42]:
points_info.shape

(711, 3)

In [222]:
points_face.shape

(8726,)

In [6]:
R2 = R[:, :, :-1,:-1]
R2.shape

(8, 45, 3, 3)

In [6]:
import torch
W = torch.tensor(W).cuda()

/home/ivenwu/miniconda3/envs/lietorch/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [93]:
old_mesh = torch.tensor(points_info).float().cuda()
# W1 = (W - W.min()) / (W.max() - W.min())
W1 = FF.softmax(W)
W1 = (W1 / (W1.sum(1, keepdim=True).detach()))
W1.shape

/home/ivenwu/miniconda3/envs/lietorch/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  This is separate from the ipykernel package so we can avoid doing imports until


torch.Size([8315, 45])

In [17]:
R = torch.tensor(R).cuda()

In [11]:
(W1.unsqueeze(2) * (R[0]@old_mesh.T).permute(2, 0, 1)).permute(1, 0, 2).sum(0).shape

TypeError: unsupported operand type(s) for @: 'numpy.ndarray' and 'Tensor'

In [235]:
new_R = SE3.InitFromVec(p1)
R = new_R.matrix()

In [8]:
R1 = R.matrix()

In [14]:
R.shape

torch.Size([8, 30, 4, 4])

In [81]:
new_R = SE3.InitFromVec(p1)
R = new_R.matrix()
wbx_result = []
for r in R:
    bx = (r@old_mesh.T).permute(2, 0, 1)
    # W = W.unsqueeze(1)
    wbx = W1.unsqueeze(2) * bx

    wbx =  wbx.permute((1,0,2))
    wbx = wbx.sum(0).detach().cpu().numpy()
    wbx_result.append(wbx)

In [77]:
wbx_result0 = wbx_result

In [57]:
wbx = wbx_result.detach().cpu().numpy()[:, :-1]
wbx.shape

AttributeError: 'list' object has no attribute 'detach'

In [128]:
W3.unsqueeze(2) * old_mesh.unsqueeze(1).repeat(1, 45, 1)for wbx_index,single_wbx in enumerate(wbx_result):
    # single_wbx = single_wbx.detach().cpu().numpy()
    print("this is index ",wbx_index)
    print(np.sum((all_info[wbx_index]-single_wbx)**2))
    # for gt in all_info:
    #     print(np.sum((gt-single_wbx)**2))

this is index  0
21.36908918353724
this is index  1
128.30545293720138
this is index  2
373.0977968274271
this is index  3
771.560108692853
this is index  4
1293.8557314639206
this is index  5
1870.6729752832866
this is index  6
2412.4298995824365
this is index  7
2813.369257814287


In [263]:
W1 = W1.detach().cpu().numpy()

In [264]:
W1 = torch.tensor(W1)

In [63]:
color_list = []
number_list = [0,0.2,0.4,0.6,0.8,1]
for i in range(B):
    color = random.sample(number_list,3)
    color_list.append(color)

In [64]:
color_array = np.array(color_list)

In [7]:
color_array = np.array([[0.8, 0.4, 0.2],
       [0.6, 0.8, 1. ],
       [0.6, 0.2, 0.4],
       [1. , 0.6, 0.8],
       [0.2, 0.4, 0. ],
       [0.2, 0.4, 0.8],
       [0.4, 1. , 0.6],
       [1. , 0.4, 0. ],
       [0.6, 0.2, 0. ],
       [0.2, 1. , 0.4],
       [0.6, 0.2, 0.4],
       [0.6, 0.4, 0. ],
       [0.4, 0.6, 0.8],
       [0.8, 0. , 1. ],
       [0.4, 0.6, 0.2],
       [0.6, 1. , 0.2],
       [0. , 0.2, 0.4],
       [0.2, 0.8, 1. ],
       [0.4, 0.2, 1. ],
       [0.8, 0.4, 1. ],
       [0.8, 1. , 0. ],
       [1. , 0. , 0.6],
       [0.2, 1. , 0. ],
       [0.6, 0.2, 0.8],
       [1. , 0.6, 0.4],
       [0.4, 0. , 1. ],
       [0.2, 1. , 0.4],
       [0.2, 0.4, 0.8],
       [1. , 0. , 0.8],
       [1. , 0.8, 0.4]])

In [60]:
W_new = W1.clone()

In [28]:
W_new.sum()

tensor(3198.9993, device='cuda:0')

In [59]:
with torch.no_grad():
    for i in range(W1.shape[0]):
        max_idx = torch.topk(W1[i], 3)[1]
        for j in range(B):
            if j not in max_idx:
                W1[i, j] = 0
        W_new[i] = W1[i] / W1[i].sum()

NameError: name 'W_new' is not defined

In [61]:
W_c = W_new.detach().cpu().numpy()

In [13]:
(W_c@color_array).shape

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 30 is different from 40)

In [39]:
pcd_test = o3d.geometry.PointCloud()
pcd_test.points = o3d.utility.Vector3dVector(points_info)
# pcd_test.colors = o3d.utility.Vector3dVector(W_c@color_array)
pcd_test.normals = o3d.utility.Vector3dVector(points_normal)


In [40]:
distances = pcd_test.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = avg_dist 

In [41]:
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
pcd_test,o3d.utility.DoubleVector([radius, radius * 2]))

In [42]:
bpa_mesh.triangles = o3d.utility.Vector3iVector(points_face-1)

In [43]:
np.array(bpa_mesh.triangles).shape

(8804, 3)

In [44]:
o3d.visualization.draw_geometries([bpa_mesh])

In [89]:
len(wbx_result)

20

In [ ]:
import open3d as o3d
pcd = o3d.geometry.PointCloud()
# pcd2 = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(wbx_result[-1][:, :-1])
# pcd.points = o3d.utility.Vector3dVector(points_info)
pcd.colors = o3d.utility.Vector3dVector(W_c@color_array)
# pcd.normals = o3d.utility.Vector3dVector(normals_info)
# pcd2.paint_uniform_color([0, 0, 1])
o3d.visualization.draw_geometries([pcd])


In [83]:
import open3d as o3d
pcd = o3d.geometry.PointCloud()
# pcd2 = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_info)
# pcd2.points = o3d.utility.Vector3dVector(eigvector_array)
pcd.colors = o3d.utility.Vector3dVector(W_c@color_array)
# pcd2.paint_uniform_color([0, 0, 1])
o3d.visualization.draw_geometries([pcd])


RuntimeError: 

In [19]:
W3 = W1[:, -2]
# W2 = W3.detach().cpu().numpy()

In [21]:
onet = torch.zeros((8315, 2)).cuda()
# ff = torch.randn((8315,2)).cuda()
W2 = torch.cat([W3.unsqueeze(1), onet], dim=1)
W2.shape

torch.Size([8315, 3])

In [22]:
W2 = W2.detach().cpu().numpy()

In [212]:
W2.shape

(8315,)

In [145]:
import openmesh

In [223]:
mesh11 = openmesh.PolyMesh()
vv = []
for i in points_info:
    vv.append(mesh11.add_vertex(i))

In [224]:
for j in points_face:

    if len(j) ==3:
        mesh11.add_face([vv[j[0]-1],vv[j[1]-1],vv[j[2]-1]])
    elif len(j) ==4:
        mesh11.add_face([vv[j[0]-1],vv[j[1]-1],vv[j[2]-1],vv[j[3]-1]])


In [226]:
openmesh.write_mesh(mesh11, "bunny1111.obj")

TypeError: write_mesh(): incompatible function arguments. The following argument types are supported:
    1. (filename: str, mesh: openmesh.TriMesh, binary: bool = False, msb: bool = False, lsb: bool = False, swap: bool = False, vertex_normal: bool = False, vertex_color: bool = False, vertex_tex_coord: bool = False, halfedge_tex_coord: bool = False, edge_color: bool = False, face_normal: bool = False, face_color: bool = False, color_alpha: bool = False, color_float: bool = False) -> None
    2. (filename: str, mesh: openmesh.PolyMesh, binary: bool = False, msb: bool = False, lsb: bool = False, swap: bool = False, vertex_normal: bool = False, vertex_color: bool = False, vertex_tex_coord: bool = False, halfedge_tex_coord: bool = False, edge_color: bool = False, face_normal: bool = False, face_color: bool = False, color_alpha: bool = False, color_float: bool = False) -> None

Invoked with: <openmesh.PolyMesh object at 0x7f2207309f30>, 'bunny1111.obj'

In [165]:
points_face.max()

8315

In [56]:
R[0, 0, :-1, :-1]@(R[0, 0, :-1, :-1].T)

tensor([[ 1.0000e+00, -1.5586e-08,  1.6673e-08],
        [-1.5586e-08,  1.0000e+00, -2.3843e-08],
        [ 1.6673e-08, -2.3843e-08,  1.0000e+00]], device='cuda:0',
       grad_fn=<MmBackward>)

In [86]:
np.vstack([wbx_result[3][:, :-1], Bone_location[3]])

array([[ 0.60822886,  2.6521692 ,  0.7332895 ],
       [ 0.63643765,  2.6594446 ,  0.63895345],
       [ 0.66882116,  2.567572  ,  0.6456163 ],
       ...,
       [-0.85560316, -1.5839747 , -0.34642002],
       [-0.56195986,  1.6341548 ,  1.198669  ],
       [ 0.34078336, -0.55604595, -0.81593436]], dtype=float32)

In [76]:
Bone_location[3].shape

(45, 3)

In [78]:
wbx_result[3][:, :-1].shape

(8315, 3)

In [88]:
X = np.array([[1, 2], [1, 4], [1, 0],
...               [10, 2], [10, 4], [10, 0]])

In [89]:
X.shape

(6, 2)

In [118]:
center, indexes = kmeans_plusplus(old_mesh[:, :-1].detach().cpu().numpy(), n_clusters=10, random_state=0)

In [98]:
t

(45, 3)

In [104]:
np.save("R.npy", R.detach().cpu().numpy())

In [105]:
np.save("W.npy", W.detach().cpu().numpy())

In [9]:
W = np.load('W.npy')

In [10]:
R = np.load('R.npy')

In [24]:
R[0]

tensor([[[ 6.6551e-01,  5.5304e-01, -5.0125e-01,  7.7804e-01],
         [-4.2060e-01,  8.3265e-01,  3.6026e-01, -3.1423e-01],
         [ 6.1661e-01, -2.8933e-02,  7.8674e-01,  3.8223e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[ 9.9372e-01, -1.0652e-01, -3.4407e-02,  1.7831e+00],
         [ 1.1189e-01,  9.5393e-01,  2.7840e-01,  2.1400e-01],
         [ 3.1677e-03, -2.8050e-01,  9.5985e-01, -5.5811e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[-5.4936e-02,  3.2795e-01, -9.4310e-01,  5.8502e-01],
         [-4.7454e-01,  8.2246e-01,  3.1365e-01, -9.7908e-01],
         [ 8.7852e-01,  4.6477e-01,  1.1044e-01,  2.0641e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[ 5.6366e-01,  6.3519e-01, -5.2803e-01, -5.1763e-01],
         [-8.2434e-01,  4.7319e-01, -3.1073e-01,  6.6134e-01],
         [ 5.2489e-02,  6.1043e-01,  7.9033e-01, -1.2407e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00, 

In [5]:
R_tmp = R[0]

In [7]:
eigvector_list = []
for i in range(R_tmp.shape[0]):
    (evals,evecs) = torch.eig(torch.tensor(R_tmp[i]),eigenvectors=True)
#     print(i)
#     print(evals)
#     print(evecs)
    flag =  0
    for val_index in range(len(evals)):
        
        if torch.abs(evals[val_index][0]-1)<1e-5 and torch.abs(evals[val_index][1])<1e-5:

            scale_info = -100
            if torch.abs(evecs[val_index][0])+torch.abs(evecs[val_index][1])+torch.abs(evecs[val_index][2])>1e-4:
#                 print(evecs[val_index])
#                 vector_wanted = (evecs[val_index]/torch.abs(evecs[val_index][-1]))[:-1]
                vector_wanted = evecs[val_index][:-1]
#                 flag = 1
#                 if scale_info != -100:
#                     vector_wanted = vector_wanted/torch.abs(scale_info)
#                     eigvector_list.append(vector_wanted.cpu().detach().numpy())
#                     break
#             else:
#                 scale_info = evecs[val_index][-1]
#                 if flag==1:
#                     vector_wanted = vector_wanted/torch.abs(scale_info)
#                     eigvector_list.append(vector_wanted.cpu().detach().numpy())
#                     break
#                 eigvector_list.append(vector_wanted.cpu().detach().numpy())
#                 break

/home/ivenwu/miniconda3/envs/lietorch/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: torch.eig is deprecated in favor of torch.linalg.eig and will be removed in a future PyTorch release.
torch.linalg.eig returns complex tensors of dtype cfloat or cdouble rather than real tensors mimicking complex tensors.
L, _ = torch.eig(A)
should be replaced with
L_complex = torch.linalg.eigvals(A)
and
L, V = torch.eig(A, eigenvectors=True)
should be replaced with
L_complex, V_complex = torch.linalg.eig(A) (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:2897.)
  This is separate from the ipykernel package so we can avoid doing imports until


In [147]:
eigvector_list = []
for i in range(R_tmp.shape[0]):

    (evals,evecs) = torch.linalg.eig(R_tmp[i])
    for j in range(4):
        if torch.abs(evecs[j].real.max()) > 1e-5 and torch.abs(evecs[j].imag.max()) < 1e-5:
            eigvector_list.append(evecs[j])
    # print(evals, evecs)

In [8]:
eigvector_list

[]

In [133]:
R_tmp[i].shape

torch.Size([4, 4])

In [99]:
eigvector_array = np.array(eigvector_list)

In [104]:
eigvector_array

array([[-2.08931750e+05, -9.44651125e+05,  1.12544150e+06],
       [-3.24681225e+06, -1.43762031e+05,  8.98924188e+05],
       [ 1.95427598e+04, -5.86003062e+05,  3.64042188e+05],
       [-5.96894375e+05, -1.18165962e+06,  2.50963225e+06],
       [-1.24324430e+05,  4.42932750e+06,  4.41323500e+06],
       [ 1.31386288e+06, -3.10073656e+05, -5.37888375e+05],
       [-1.26899762e+06,  9.39687400e+06,  7.66483850e+06],
       [-9.65893750e+05,  0.00000000e+00,  3.28692266e+04],
       [-2.63832363e+04, -5.67883438e+05,  6.76968875e+05],
       [ 4.71411600e+06,  0.00000000e+00,  1.49484412e+06],
       [ 9.45399625e+05, -1.65568438e+06,  2.87590175e+06],
       [-5.24790875e+05,  1.72209975e+06,  1.08762620e+07],
       [-8.25391094e+04,  3.47957688e+05,  2.88241406e+05],
       [-1.15535425e+06,  7.71927500e+06,  5.02589250e+06],
       [-9.30507812e+05, -2.15668350e+06,  3.12389625e+06],
       [-2.89633725e+06, -4.90623450e+06,  6.42465350e+06],
       [-8.08298650e+06, -9.63290750e+05

In [103]:
for k in range(eigvector_array.shape[0]):
    
    print(eigvector_array[k].max(),eigvector_array[k].min())

1125441.5 -944651.1
898924.2 -3246812.2
364042.2 -586003.06
2509632.2 -1181659.6
4429327.5 -124324.43
1313862.9 -537888.4
9396874.0 -1268997.6
32869.227 -965893.75
676968.9 -567883.44
4714116.0 0.0
2875901.8 -1655684.4
10876262.0 -524790.9
347957.7 -82539.11
7719275.0 -1155354.2
3123896.2 -2156683.5
6424653.5 -4906234.5
-963290.75 -8082986.5
8315559.5 29267.422
266241.4 -2776276.0
7286642.5 0.0
4245924.0 -517617.72
317961.75 -934659.5
9409278.0 -515592.22
502299.16 -2235584.8
5599194.0 1259836.9
889088.3 -4156421.2
1063102.4 -114584.5
6152423.5 -237797.12
-401868.62 -1901944.4
-84895.055 -1103657.4
458805.28 -213043.7
2136201.5 578177.0
1281208.0 -15912.146
1737261.4 -8719609.0
8410769.0 -2069724.2
10213309.0 -1847051.1
78522.86 0.0
198893.73 -49068.426
1405885.1 -602139.7
4323145.0 -213344.0
9354421.0 -651071.4
6311035.0 -325663.12
212339.86 -1727798.6
7500047.5 -1767407.0
335404.25 -224781.38


In [29]:
(evals,evecs) = torch.eig(R[0][1],eigenvectors=True)

In [30]:
evals

tensor([[ 1.0000,  0.0000],
        [ 0.9537,  0.3006],
        [ 0.9537, -0.3006],
        [ 1.0000,  0.0000]], device='cuda:0')

In [31]:
evecs

tensor([[-9.2959e-01, -4.1160e-02,  2.5737e-01,  9.2959e-01],
        [-6.2496e-02,  7.0572e-01,  0.0000e+00,  6.2496e-02],
        [ 3.6326e-01,  1.6084e-02,  6.5861e-01, -3.6326e-01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  2.8631e-07]], device='cuda:0')

In [28]:
R[0][0]

tensor([[ 0.6655,  0.5530, -0.5012,  0.7780],
        [-0.4206,  0.8327,  0.3603, -0.3142],
        [ 0.6166, -0.0289,  0.7867,  0.3822],
        [ 0.0000,  0.0000,  0.0000,  1.0000]], device='cuda:0')

In [124]:
for j in range(45):
    R[0] 

torch.Size([8, 45, 4, 4])

In [ ]:
def train(old_info,target_info, n_steps=1000, lr_init=0.2):
    """ Riemannian Gradient Descent """

    N = points_info.shape[0]
    F = target_info.shape[0]
    B = 45  ### 这个肯定可以通过读pkl得到


    old_mesh = torch.tensor(old_info).float().cuda().detach()
    
    # bp()
    # ones = torch.ones(old_mesh.shape[0], 1).cuda()
    # x = torch.cat([old_mesh, ones], dim=1)


    target_mesh = torch.tensor(target_info, requires_grad=False).float().cuda().detach()

    # bp()
    # W = torch.randint(10,(N,B))
    # W = torch.randn((N,B)).cuda()
    # W = np.random.randint(10,size=(N,B))
    # bp()
    # W = SO3(torch.from_numpy(W).float().cuda())
    # W = torch.zeros((N, B), dtype=torch.float, requires_grad=True, device="cuda")
    W = torch.randn((N, B), requires_grad=True, device="cuda")
    W2 = torch.randn((N, B), requires_grad=True, device="cuda")
    # W = torch.tensor(W0.float(), requires_grad=True, device="cuda")
    # W = W * 0
    # bp()
    # W = LieGroupParameter(W)
    # bp()
    # R = torch.randint(10,(B,3,3))laplacian = torch.zeros([self.nv, self.nv]).astype(np.float32)

    # bp()
    # random quaternion
    p1 = torch.randn((F, B, 7), requires_grad=True, device="cuda")
    # W = W.cuda()
    # p1 = p1.cuda()
    # p1 = p1 / p1.norm(dim=-1, keepdim=True)
    # create SO3 object from quaternion (differentiable w.r.t q)
    
    # 4x4 transformation matrix (differentiable w.r.t R)
    


    optimizer = optim.Adam([W, p1],lr=1e-3)
    chamLoss = chamfer3D.dist_chamfer_3D.chamfer_3DDist()
    for i in range(n_steps):

        optimizer.zero_grad()
        loss = 0
        f_idxs = random.sample(range(F), F)
        
        for f in f_idxs:
            T = torch.randn(B, 4, 4)
            for b in range(B):
                # R = SO3.InitFromVec(p1[f, b])
                R = SE3.InitFromVec(p1[f, b])
                # bp()
                T[b] = R.matrix()
            T = T.cuda()     
            # bp()      
            # bp() 
            # W1 = W
            # W1 = (W - W.min()) / (W.max() - W.min())
            W1 = FF.softmax(W*10)
            W1 = (W1 / (W1.sum(1, keepdim=True).detach()))
            W2[W1 < 0.1] = 0
            
            # ones = torch.ones(old_mesh.shape[0], 1).cuda()
            # x = torch.cat([old_mesh, ones], dim=1)
            x = old_mesh
        
        
            bx = (T@x.T).permute(2, 0, 1)
            wbx = W1.unsqueeze(2) * bx
            # bp()
            wbx =  wbx.permute((1,0,2))
            # print("Wbx", wbx.shape)
            # wbx = wbx.sum(0, keepdim=True)[:, :, :-1]
            wbx = wbx.sum(0, keepdim=True)
            # loss_init = chamLoss(target_mesh[f][None],wbx)
            # loss += loss_init[0].sum()

            loss += torch.sum((target_mesh[f]-wbx)**2) + torch.sum(torch.abs(W))

#             loss.backward()
            print("loss",loss)
            # bp()
        # bp()
        loss.backward()
        # print("this is loss for epoch {} : {} ".format(i,loss.detach().cpu().numpy()))
        print("this is loss for epoch {} : {} ".format(i,loss / F))
        # print("p1: ", p1)
        optimizer.step()


    new_R = SE3.InitFromVec(p1)
    R = new_R.matrix()

    # W1 = (W - W.min()) / (W.max() - W.min())
    # W1 = (W1 / (W1.sum(1, keepdim=True).detach()))
    # bp()
    # return R.detach(), W1.detach()
    return W.detach(),R.detach()